# 3. Thermodynamic variability analysis (TVA)

There exists multiple flux states that define the same optimum. TVA predicts the ranges of variables such as metabolic fluxes, Gibbs energy of reactions and metabolite concentrations by taking into account of thermodynamic uncertainities using multivariate approach.

As described previously, you can solve the TVA problem on either the MILP or MIQCP. Although, MILP is faster compared to solving MIQCP, MIQCP is thermodynamically consistent as they drawn from multivariate distribution. 

There are three different functions for running TVA in multiTFA. One for the MILP, the remaining two for MIQCP, depending on the solver you use (either Gurobi or Cplex).

By default, TVA is performed on all the variables in a model unless `variable_list` parameter is specified. It is a `list` of variable names you want to perform TVA on. For example reactons id's or names of Gibbs energy variables etc.

The example below demonstrates how to perform TVA on *E. coli* core model with constraints described in previous sections. We will use `cplex` for our calculations. In our observations `Cplex` ourperforms `Gurobi`. 


In [1]:
from multitfa.test_model import load_test_data
tfa_model = load_test_data() # Load the test model

from multitfa import analysis
dg_vars = [rxn.delG_forward.name for rxn in tfa_model.reactions if rxn.id not in tfa_model.Exclude_reactions] # Gibbs energy variable names


Fragments already downloaded
Fragments already downloaded
Fragments already downloaded
Using license file /home/vishnu/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /tmp/tmp4kxnn3mw.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


Now let us perform TVA on `tfa_model` using univariate approch and calculate ranges on reaction flux variables

In [2]:
tfa_model.solver = 'cplex'
ranges_box = analysis.variability(tfa_model, variable_list = dg_vars) # TVA using MILP
print(ranges_box)

               minimum    maximum
dG_PFK      -65.527361  26.205939
dG_PFL      -72.197674  33.074152
dG_PGI      -19.085713  24.931203
dG_PGK        0.000000  59.219274
dG_PGL      -40.671665  28.293843
...                ...        ...
dG_ME2      -70.983610  57.688660
dG_NADH16  -194.698524 -61.742679
dG_NADTRHD  -58.319625  57.785742
dG_NH4t     -13.742207  32.066015
dG_PDH     -116.749219  48.618586

[72 rows x 2 columns]


 `ranges_box` is a `pandas DataFrame` which can be readily exported to a csv. 

In [3]:
ranges_box.to_csv('ranges.csv')

Now lets solve the TVA with multivariate approach using MIQCP. There are two functions one when using `Gurobi` and another for `Cplex`.

In [4]:
ranges_MIQC_cplex = analysis.variability_legacy_cplex(tfa_model,variable_list = dg_vars) # TVA using MIQCP with cplex
print(ranges_MIQC_cplex)


Selected objective sense:  MINIMIZE
Selected objective  name:  c5b998a4-5b9b-11eb-9753-509a4c43d0a7
Selected RHS        name:  rhs
Selected bound      name:  bnd

Selected objective sense:  MINIMIZE
Selected objective  name:  c5b998a4-5b9b-11eb-9753-509a4c43d0a7
Selected RHS        name:  rhs
Selected bound      name:  bnd
                 minimum    maximum
dG_PFK     -6.349569e+01  24.174265
dG_PFL     -7.084883e+01  31.725309
dG_PGI     -1.940986e+01  25.255346
dG_PGK      1.363638e-10  56.021188
dG_PGL     -4.961171e+01  37.233887
...                  ...        ...
dG_ME2     -8.241713e+01  69.129418
dG_NADH16  -2.107758e+02 -33.736290
dG_NADTRHD -4.100270e+01  23.163226
dG_NH4t    -9.670293e+00  27.994102
dG_PDH     -1.200410e+02  31.809464

[72 rows x 2 columns]


Similarly, if you need to use `Gurobi`, you should use `variability_legacy_gurobi` function. Optionally, if you wish to warm start the optimization, you should specify `warm_start` parameter.

Please note, runnig TVA on genome scale model is computationally expensive, especially using MIQC problems. We have observed that relaxing the `Gap` parameter helped achieving faster run times. 

## 3.1 Alternative way to solve MIQC

When `Gurobi` or `Cplex` is not available, and if you wanted to run MIQC problem, we developed a sampling based approach to solve the MIQCP. We sample the uncertainity variables on the surface of the ellipsoid and solve the subsequent MILP problem. This can be solved using MILP solvers. The exit criterion of the sampler can be chosen as either

(1) the number of samples since last improvement or 
(2) a fixed number of samples followed by use of a generalized extreme value distribution to infer the maximum value.

In [5]:
gev_ranges, gev_samples = analysis.gev_sampling(tfa_model, variable_list=dg_vars, cutoff = 100)

In [6]:
print(gev_samples)

            minimum        maximum     minimum    maximum     minimum  \
dG_PFK          0.0  6.932509e-310  -59.208994  16.119795  -57.244604   
dG_PFL          0.0  6.932508e-310  -55.968738  19.360052  -55.302620   
dG_PGI          0.0  6.932508e-310  -15.750196  21.914199  -15.050496   
dG_PGK          0.0  6.932508e-310    0.000000  49.796896    0.000000   
dG_PGL          0.0  6.932508e-310  -34.714650  21.781942  -35.021775   
...             ...            ...         ...        ...         ...   
dG_ME2          0.0  6.932514e-310  -57.435739  36.725248  -50.295732   
dG_NADH16       0.0  6.932507e-310 -172.122538 -96.793749 -158.761021   
dG_NADTRHD      0.0  6.932507e-310  -38.011098  20.720051  -37.953588   
dG_NH4t         0.0  6.932514e-310   -9.670293  27.994102   -9.670293   
dG_PDH          0.0  6.932514e-310  -91.000549  21.992636  -84.902043   

              maximum     minimum    maximum     minimum    maximum  ...  \
dG_PFK      18.084186  -57.289776  18.039014  -

`gev_samples` is the alternate set of Gibbs energy of reactions and `gev_ranges` is the max-min ranges of Gibbs energy of reactions. By specifying `cutoff` parameter, we specify how many samples we use to train the extreme value distribution (default, 1000). The more samples the better. 

Alternatively, you could choose to use the other exit crieteria where if we don't see improvement since a defined number of samples.

In [7]:
cutoff_ranges, cutoff_samples, num_samples = analysis.cutoff_sampling(tfa_model, cutoff=10, variable_list=dg_vars)

Here, `cutoff_ranges`, `cutoff_samples`, `num_samples` represents max-min Gibbs energy ranges of reactions, alternate Gibbs energy of reactions and num of samples to achieve the exit.

In [8]:
print(cutoff_samples)

            minimum    maximum     minimum    maximum     minimum    maximum  \
dG_PFK          0.0  20.035991  -57.252994  18.075796  -57.636151  17.692639   
dG_PFL          0.0  21.728102  -59.638944  15.689845  -56.113491  19.215298   
dG_PGI          0.0  22.695646  -15.477020  22.187375  -16.170388  21.494007   
dG_PGK          0.0  52.086147    0.000000  51.281604    0.000000  50.618981   
dG_PGL          0.0  29.168896  -33.188921  23.307671  -36.148757  20.347835   
...             ...        ...         ...        ...         ...        ...   
dG_ME2          0.0  49.223198  -54.162524  39.998463  -56.020423  38.140564   
dG_NADH16       0.0 -55.540377 -164.345338 -89.016549 -161.914787 -86.585998   
dG_NADTRHD      0.0  21.943241  -38.643810  19.470017  -38.697513  17.149985   
dG_NH4t         0.0  27.993973   -9.670293  27.994102   -9.670293  27.994102   
dG_PDH          0.0  31.727693  -93.739895   7.605108  -94.594519   6.201065   

               minimum    maximum     m

## 3.2 Network embeded thermodynamics (NET)

By using the TVA apparoches demonstrated above, one can perform NET type of analysis. For example, we can fix the directionalities of all the reactions and sample for the metabolite concentrations. 

Reaction directionalities can be fixed by adjusting the lower or upper bounds of the reactions.

In [9]:
atps = tfa_model.reactions.get_by_id('ATPS4r')
print("lower bound is {} and upper bound is {}\n".format(atps.lower_bound, atps.upper_bound))
atps.lower_bound = 0
print("updated lower and upper bounds are {} \t {}\n".format(atps.lower_bound, atps.upper_bound))

lower bound is -1000.0 and upper bound is 1000.0

updated lower and upper bounds are 0 	 1000.0



We can access the metabolite concentration variables as shown below,

In [10]:
concentration_vars = [var.name for var in tfa_model.variables if var.name.startswith('lnc_')]
tfa_model.solver = 'cplex'
concentration_ranges = analysis.variability_legacy_cplex(tfa_model,variable_list = concentration_vars) # TVA using MIQCP with cplex
print(concentration_ranges)


Selected objective sense:  MINIMIZE
Selected objective  name:  c5b998a4-5b9b-11eb-9753-509a4c43d0a7
Selected RHS        name:  rhs
Selected bound      name:  bnd
                minimum   maximum
lnc_glc__D_e -11.512925 -3.912023
lnc_gln__L_c -11.512925 -3.912023
lnc_gln__L_e -11.512925 -3.912023
lnc_glu__L_c -11.512925 -3.912023
lnc_glu__L_e -11.512925 -3.912023
...                 ...       ...
lnc_fru_e    -11.512925 -3.912023
lnc_fum_c    -11.512925 -3.912023
lnc_fum_e    -11.512925 -3.912023
lnc_g3p_c    -11.512925 -3.912023
lnc_g6p_c    -11.512925 -3.912023

[72 rows x 2 columns]
